# `CLOUD.jl` - 2D linear advection example

In [10]:
using CLOUD, OrdinaryDiffEq

Define the physical problem

In [11]:
θ = π/4 # wave angle
a = sqrt(2) # wave speed
A = 1.0  # amplitude
L = 1.0  # domain length
k = (2*π/L, 2*π/L)  # wave number
T = 1.0/(a*max(abs(cos(θ)),abs(cos(θ)))) # end time for one period

initial_data = InitialDataSine(A,k)
conservation_law = LinearAdvectionEquation((a*cos(θ),a*sin(θ)));

Set discretization parameters

In [12]:
M = 4
p = 4
p_map = 4
form = WeakConservationForm(StandardMapping(),
    LaxFriedrichsNumericalFlux())
strategy = Lazy()

ode_algorithm = CarpenterKennedy2N54()
dt = 0.1*(L/M)/(a*(2*p+1));
write_interval = floor(Int, T/(dt*10));

## Collocated DGSEM on quadrilaterals

Set up a `DGSEM` scheme with LG quadrature on quadrilaterals of degree `p` with `M` edges in each direction

In [13]:
reference_approximation =ReferenceApproximation(
    DGSEM(p), Quad(),
    volume_quadrature_rule=LGQuadrature(),
    facet_quadrature_rule=LGQuadrature(),
    mapping_degree=p_map, N_plot=50)

mesh = warp_mesh(uniform_periodic_mesh(
    reference_approximation.reference_element, 
    ((0.0,L),(0.0,L)), (M,M)), 
    reference_approximation.reference_element, 0.1)

spatial_discretization = SpatialDiscretization(mesh, 
    reference_approximation)

results_path = save_project(conservation_law,
     spatial_discretization, initial_data, form, (0.0, T), Lazy(),
     "results/advection_2d_dgsem/", overwrite=true, clear=true)

"results/advection_2d_dgsem/"

Run the solver

In [14]:
ode_problem = semidiscretize(conservation_law,
    spatial_discretization,
    initial_data, 
    form,
    (0.0, T),
    strategy)

save_solution(ode_problem.u0, 0.0, results_path, 0)

CLOUD_reset_timer()
sol = solve(ode_problem, ode_algorithm, adaptive=false, dt=dt,
    save_everystep=false, callback=save_callback(results_path, write_interval))
CLOUD_print_timer()

save_solution(last(sol.u), last(sol.t), results_path, "final")

 ────────────────────────────────────────────────────────────────────────────────
                                        Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:         77.8μs / 823292.5%        10.6KiB / 6257209.6%  

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 eval residual          40.8k    346ms   54.0%  8.48μs    293MiB   45.4%  7.34KiB
   volume terms         40.8k    149ms   23.2%  3.65μs    156MiB   24.1%  3.91KiB
   facet terms          40.8k   74.9ms   11.7%  1.84μs   86.6MiB   13.4%  2.17KiB
   mass matrix solve    40.8k   57.5ms    9.0%  1.41μs   19.9MiB    3.1%     512B
 gather ext state       40.8k    133ms   20.8%  3.26μs    131MiB   20.3%  3.29KiB
 extrap solution        40.8k   78.2ms   12.2%  1.92μs   71.0MiB   11.0%  1.78KiB
 eval num flux 

Calculate the integral L<sup>2</sup> error norm and conservation/energy error

In [15]:
error_analysis = ErrorAnalysis(results_path, conservation_law, 
    spatial_discretization)
conservation_analysis = PrimaryConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)
energy_analysis = EnergyConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)

println("L2 error:")
println(analyze(error_analysis, last(sol.u), initial_data)...)
println("Conservation (initial/final/diff):")
println(analyze(conservation_analysis)...)
println("Energy (initial/final/diff):")
println(analyze(energy_analysis)...)

L2 error:
0.00068216127441316
Conservation (initial/final/diff):
[-1.108594540819019e-8][-1.1085946192285201e-8][-7.840950111415168e-16]
Energy (initial/final/diff):
[0.12499998811286066][0.12497973419588697][-2.025391697368928e-5]


## Modal DG scheme on triangles

Set up a `DGMulti` scheme on triangles of degree `p` with `M` edges in each direction

In [16]:
reference_approximation = ReferenceApproximation(
    DGMulti(p), Tri(), mapping_degree=p_map, N_plot=10)

mesh = warp_mesh(uniform_periodic_mesh(
    reference_approximation.reference_element, 
    ((0.0,L),(0.0,L)), (M,M)), 
    reference_approximation.reference_element, 0.1)

spatial_discretization = SpatialDiscretization(mesh, 
    reference_approximation)

results_path = save_project(conservation_law,
     spatial_discretization, initial_data, form, (0.0, T), Lazy(),
     "results/advection_2d_dgmulti/", overwrite=true, clear=true)

"results/advection_2d_dgmulti/"

Run the solver

In [17]:
ode_problem = semidiscretize(conservation_law,
    spatial_discretization,
    initial_data, 
    form,
    (0.0, T),
    strategy)

save_solution(ode_problem.u0, 0.0, results_path, 0)

CLOUD_reset_timer()
sol = solve(ode_problem, ode_algorithm, adaptive=false, dt=dt,
    save_everystep=false, callback=save_callback(results_path, write_interval))
CLOUD_print_timer()

save_solution(last(sol.u), last(sol.t), results_path, "final")

 ────────────────────────────────────────────────────────────────────────────────
                                        Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:         77.7μs / 1213946.7%      10.6KiB / 10385157.0%  

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 eval residual          81.6k    524ms   55.5%  6.41μs    453MiB   42.3%  5.69KiB
   mass matrix solve    81.6k    208ms   22.0%  2.55μs    186MiB   17.3%  2.33KiB
   volume terms         81.6k    187ms   19.9%  2.30μs    206MiB   19.2%  2.58KiB
   facet terms          81.6k   28.2ms    3.0%   346ns   13.7MiB    1.3%     176B
 eval flux              81.6k    143ms   15.2%  1.76μs   69.8MiB    6.5%     896B
 eval num flux          81.6k   93.0ms    9.9%  1.14μs    167MiB   15.6%  2.09KiB
 extrap solutio

Calculate the integral L<sup>2</sup> error norm and conservation/energy error

In [18]:
error_analysis = ErrorAnalysis(results_path, conservation_law, 
    spatial_discretization)
conservation_analysis = PrimaryConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)
energy_analysis = EnergyConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)

println("L2 error:")
println(analyze(error_analysis, last(sol.u), initial_data)...)
println("Conservation (initial/final/diff):")
println(analyze(conservation_analysis)...)
println("Energy (initial/final/diff):")
println(analyze(energy_analysis)...)

L2 error:
0.0034594895345242908
Conservation (initial/final/diff):
[-1.4530229013823975e-7][-1.4530229063480435e-7][-4.96564594998361e-16]
Energy (initial/final/diff):
[0.12499475017905026][0.12485531367220426][-0.0001394365068460024]
